In [ ]:
import numpy as np
import pandas as pd
from time import sleep
#pip install googlesearch-python
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import os

In [ ]:
logging.basicConfig(filename='errorfile.log' , level = logging.INFO)

In [ ]:
def init_driver():
    # Initialize chrome driver
    browser = webdriver.Chrome(ChromeDriverManager().install())
    # browser = webdriver.Chrome('give path to your file here')
    return browser

def get_pause():
    return np.random.choice(range(5,10))

def fetchCompaniesFromExcel():
    try:
        # open companies.xlsx file and save all companies in the form of list
        df = pd.read_excel("companies.xlsx")
        mylist =  df.reset_index()[['Name']].values.tolist()
        return mylist
    except Exception as e:
        logging.error('Error occured in function fetchCompaniesFromExcel'+  str(e))

def SearchCompany(browser, companyName):
    # open Glassdoor and search for particular company through automated browser
    try:
        company = browser.find_element_by_id("KeywordSearch")  
        location = browser.find_element_by_id("LocationSearch") 
        #sleep(1)
        company.send_keys(companyName)  
        #sleep(1)
        location.clear()
        browser.find_element_by_class_name('gd-btn-mkt').click()
        browser.implicitly_wait(10)
        curr_guid = browser.current_window_handle;
        browser.switch_to_window(curr_guid);
        allGuids=  browser.window_handles;
        for i in allGuids:
            if(i != curr_guid):
                browser.switch_to_window(i);
        print('Close the browser')   

        # we will create a string of company data separated by '#' and then parse the string
        companyOverview = ''
        #sleep(1)
        try:   
            # finding company link here from the results of the search on Glassdoor website
            elem =  browser.find_element_by_xpath('//*[@id="MainCol"]/div[1]/div[2]/div[1]/div[2]/div[1]/a')
            print(elem)
            elem.click() 
        except:
            print("company link not found")
        
        tutorial_soup = BeautifulSoup(browser.page_source, 'html.parser')
    
        try:
            # finding number of reviews here
            numReview = tutorial_soup.find("a", {"class": "reviews"}).get_text()
            print('outside if')
            print('Number of Reviews are: ', numReview)
        except:
            numReview = 0            
            print('Number of Reviews are not found')
        numReview = str(numReview).strip()
        numReview = numReview.split(" ")
        numReview = numReview[0]
        companyOverview = companyOverview +'NumReview:'+ numReview
        
        try:
            # finding number of Job Openings here
            openingsDiv = tutorial_soup.find("a", {"class": "jobs"})
            numOpenings = openingsDiv.findChild().text
            print('Number of Job Openings are: ', numOpenings)
        except:
            numOpenings = 0
            print('Number of Job Openings are not found')
        companyOverview = companyOverview +'#Numopenings:'+str(numOpenings).strip()
            
        try:
            # finding rating number here
            ratingsDiv = tutorial_soup.find("div", {"class": "v2__EIReviewsRatingsStylesV2__ratingNum"}).get_text()
            print('Rating given is: ', ratingsDiv)
        except:
            ratingsDiv = 0
            print('No rating found') 
        companyOverview = companyOverview +'#Ratings:'+str(ratingsDiv).strip()
        companyOverview = companyOverview + '#CompanyName:' + str(companyName).strip()    
            
        browser.close()
        #sleep(1)
        return companyOverview
    except:
        print('CATCH BLOCK')
        browser.close()
        companyOverview = 'NumReview:0#Numopenings:0#Ratings:0.0#'+'CompanyName:'+companyName
        return companyOverview


#fetch list of companies from excel sheet
def fetchDetails(lst):
    try:
        lstCompanies = lst
        companyDesclst = []

        for lst in lstCompanies:
            sleep(get_pause())
            website = "https://www.glassdoor.com/Reviews/index.htm"
            # Initialize the webdriver
            browser = init_driver()
            sleep(get_pause())
            browser.get(website)
            browser.maximize_window() 
            companyName = lst[0]
            sleep(get_pause())
            companyDesc = SearchCompany(browser, companyName)
            print(companyDesc) 
            companyDesclst.append(companyDesc)
        return companyDesclst 
    except Exception as e:
        logging.error('Error occured in function fetchDetails'+  str(e))    

def main():
    try: 
        lst = fetchCompaniesFromExcel()
        finalList = fetchDetails(lst)
    except Exception as e:
        logging.error('Error occured in function SearchCompany'+  str(e))    
    try:    
        lstForDataframe = []
        for item in finalList:
            tempLst = []
            desc = item.split('#')
            numReview = desc[0].split(':')
            numReview = numReview[1]
            print(numReview)
            numOpenings = desc[1].split(':')
            numOpenings = numOpenings[1]
            print(numOpenings)
            ratingsDiv = desc[2].split(':')
            ratingsDiv = ratingsDiv[1]
            companyName= desc[3].split(':')
            companyName = companyName[1]
            print(ratingsDiv)
            tempLst.append(numReview)
            tempLst.append(numOpenings)
            tempLst.append(ratingsDiv)
            tempLst.append(companyName)
            print('TempList')
            print(tempLst)
            lstForDataframe.append(tempLst)
        dfObj = pd.DataFrame.from_records(lstForDataframe, columns=['Number of reviews', 'Job Openings', 'Ratings Value', 'Company Name'])
        dfObj.to_excel("GlassdoorData.xlsx", index=False)
        return os.getcwd()+"\\"+ "GlassdoorData.xlsx"
    except Exception as e:
        logging.error('Error occured in function createExcel:'+  str(e))


In [ ]:
main()

In [ ]:
#END